In [24]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import  request
import csv
import os
import re
import requests
import webbrowser
import socket
from jiema import jiema
socket.setdefaulttimeout(1)

In [43]:
def getChessManual(owner,id_name):#获取游戏棋谱，返回谁获胜，棋谱
    url = "http://www.dpxq.com/hldcg/search/view.asp?owner="+owner+"&id="+id_name
    user_agent = "Mozilla/5.0 (X11; Linux x86_64; rv:45.0) Gecko/20100101 Firefox/45.0"
    req = request.Request(url,headers={'User-Agent': user_agent} )
    try:
        html = urlopen(req).read().decode('gb2312')
    except Exception:
        try:
            html = urlopen(req).read().decode('gbk')
        except Exception:
            html = urlopen(req).read().decode('GB18030')
        finally:
            return ("error",'')
    soup = BeautifulSoup(html, features='lxml')
    boardData = str(soup.find_all('div',id='dhtmlxq_view')[0])
    try:
        a = re.findall(r"var DhtmlXQ_movelist.*\[DhtmlXQ_movelist\](.*)\[/DhtmlXQ_movelist\]",html)[0]
        b = re.findall(r"\[DhtmlXQ_binit\](.*)\[/DhtmlXQ_binit\]",boardData)[0]
        if b=='':
            b='8979695949392919097717866646260600102030405060708012720323436383'
        chess_type =  eval('\"'+re.findall(r"\[DhtmlXQ_type\](.*)\[/DhtmlXQ_type\]",boardData)[0]+'\"')
        if chess_type=='':
            chess_type="残局"
        winType = "\""+re.findall(r"\[DhtmlXQ_result\](.*)\[/DhtmlXQ_result\]",boardData)[0]+"\""
        result_list = jiema().getMoveListString(a,b)
        if result_list[0][1]==0:
            nextType = 'r'
        else:
            nextType = 'b'
        fen = binitToFen(b)+' '+nextType
        return fen,chess_type,eval(winType),result_list
    except Exception:
        return ("error",'')

def binitToFen(binit):
    chess_board=[[" " for i in range(9)] for i in range(10)]
    chess_type=['R','N','B','A','K','A','B','N','R','C','C','P','P','P','P','P']
    for i in range(32):
        x=int(binit[i*2])
        y=int(binit[i*2+1])
        if x==9:
            continue
        if i<=15:
            ct = chess_type[i]
        else:
            ct = str.lower(chess_type[i-16])
        chess_board[y][x] = ct
    fen = ""
    for line in chess_board:
        black=0
        for index,char in enumerate(line):
            if char==' ':
                black+=1
            else:
                if black!=0:
                    fen+=str(black)
                black=0
                fen+=char
            if index==8:
                if char==' ':
                    fen+=str(black)
                fen+='/'
    return fen[:-1]

def getOwnerPage(owner,page):
    page = str(page)
    url = "http://www.dpxq.com/hldcg/search/list.asp?owner="+owner+"&page="+page
    user_agent = "Mozilla/5.0 (X11; Linux x86_64; rv:45.0) Gecko/20100101 Firefox/45.0"
    req = request.Request(url,headers={'User-Agent': user_agent} )
    try:
        html = urlopen(req).read().decode('gb2312')
    except Exception:
        try:
            html = urlopen(req).read().decode('gbk')
        except Exception:
            html = urlopen(req).read().decode('GB18030')
        finally:
            return [['','']]
    soup = BeautifulSoup(html, features='lxml')
    boardData = soup.find_all('a',{"href":re.compile(r"javascript:view\(\'owner=(.*).*id=\d*\'\)")})
    url_list = []
    for bo in boardData:
        url_action = bo['href']
        url_result = re.findall(r'javascript:view\(\'owner=(.*)&id=(.*)\'',url_action)[0]
        url_list.append([url_result[0],url_result[1]])
    return url_list

def getOwnerList(owner,page_down = 100):
    re = set()
    for i in range(page_down):
        t = getOwnerPage(owner,i)
        for uname,id_name in t:
            if id_name=='':
                continue
            re.add(id_name)
    return uname,re

def save_file(re_list,bh):
    if re_list[0]=='error':
        return
    win_type = re_list[1]
    win_who = re_list[2]
    if win_who == '和棋':
        return
    elif win_who == '':
        return
    dir_path = '棋局库/'+win_type+'/'
    if os.path.isdir(dir_path) == False:
        os.mkdir(dir_path)
    file_path = dir_path+win_who+'.csv'
    qipu_list = [str(re.findall('b\'(.*)\'',str(str(i).encode('gbk')))[0]) for i,bu in re_list[3]]
    qipu_list.insert(0,re_list[0])
    qipu_list.insert(1,bh)
    with open(file_path,'a') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(qipu_list)
        f.close()

In [ ]:
now_label = 0
for i in range(37535,1489888):
    index = str(int("10000000")+i)[1:]
    try:
        save_file(getChessManual('u',index),index)
    except Exception:
        continue
    if i%5==0:
        print(i)

37535
37540
37545
37550
37555
37560
37565
37570
37575
37580
37585
37590
37595
37600
37605
37610
37615
37620
37625
37630
37635
37640
37645
37650
37655
37660
37665
37670
37675
37680
37685
37690
37695
37700
37705
37710
37715
37720
37725
37730
37735
37740
37745
37750
37755
37760
37765
37770
37775
37780
37785
37790
37795
37800
37805
37810
37815
37820
37825
37830
37835
37840
37845
37850
37855
37860
37865
37870
37875
37880
37885
37890
37895
37900
37905
37910
37915
37920
37925
37930
37935
37940
37945
37950
37955
37960
37965
37970
37975
37980
37985
37990
37995
38000
38005
38010
38015
38020
38025
38030
38035
38040
38045
38050
38055
38060
38065
38070
38075
38080
38085
38090
38095
38100
38105
38110
38115
38120
38125
38130
38135
38140
38145
38150
38155
38160
38165
38170
38175
38180
38185
38190
38195
38200
38205
38210
38215
38220
38225
38230
38235
38240
38245
38250
38255
38260
38265
38270
38275
38280
38285
38290
38295
38300
38305
38310
38315
38320
38325
38330
38335
38340
38345
38350
38355
38360
3836

44365
44370
44375
44380
44385
44390
44395
44400
44405
44410
44415
44420
44425
44430
44435
44440
44445
44450
44455
44460
44465
44470
44475
44480
44485
44490
44495
44500
44505
44510
44515
44520
44525
44530
44535
44540
44545
44550
44555
44560
44565
44570
44575
44580
44585
44590
44595
44600
44605
44610
44615
44620
44625
44630
44635
44640
44645
44650
44655
44660
44665
44670
44675
44680
44685
44690
44695
44700
44705
44710
44715
44720
44725
44730
44735
44740
44745
44750
44755
44760
44765
44770
44775
44780
44785
44790
44795
44800
44805
44810
44815
44820
44825
44830
44835
44840
44845
44850
44855
44860
44865
44870
44875
44880
44885
44890
44900
44905
44910
44915
44920
44925
44930
44935
44940
44945
44950
44955
44960
44965
44970
44975
44980
44985
44990
44995
45000
45005
45010
45015
45020
45025
45030
45035
45040
45045
45050
45055
45060
45065
45070
45075
45080
45085
45090
45095
45100
45105
45110
45115
45120
45125
45130
45135
45140
45145
45150
45155
45160
45165
45170
45175
45180
45185
45190
45195
4520

51200
51205
51210
51215
51220
51225
51230
51235
51240
51245
51250
51255
51260
51265
51270
51275
51280
51285
51290
51295
51300
51305
51310
51315
51320
51325
51330
51335
51340
51345
51350
51355
51360
51365
51370
51375
51380
51385
51390
51395
51400
51405
51410
51415
51420
51425
51430
51435
51440
51445
51450
51455
51460
51465
51470
51475
51480
51485
51490
51495
51500
51505
51510
51515
51520
51525
51530
51535
51540
51545
51550
51555
51560
51565
51570
51575
51580
51585
51590
51595
51600
51605
51610
51615
51620
51625
51630
51635
51640
51645
51650
51655
51660
51665
51670
51675
51680
51685
51690
51695
51700
51705
51710
51715
51720
51725
51730
51735
51740
51745
51750
51755
51760
51765
51770
51775
51780
51785
51790
51795
51800
51805
51810
51815
51820
51825
51830
51835
51840
51845
51850
51855
51860
51865
51870
51875
51880
51885
51890
51895
51900
51905
51910
51915
51920
51925
51930
51935
51940
51945
51950
51955
51960
51965
51970
51975
51980
51985
51990
51995
52000
52005
52010
52015
52020
52025
5203

58030
58035
58040
58045
58050
58055
58060
58065
58070
58075
58080
58085
58090
58095
58100
58105
58110
58115
58120
58125
58130
58135
58140
58145
58150
58155
58160
58165
58170
58175
58180
58185
58190
58195
58200
58205
58210
58215
58220
58225
58230
58235
58240
58245
58250
58255
58260
58265
58270
58275
58280
58285
58290
58295
58300
58305
58310
58315
58320
58325
58330
58335
58340
58345
58350
58355
58360
58365
58370
58375
58380
58385
58390
58395
58400
58405
58410
58415
58420
58425
58430
58435
58440
58445
58450
58455
58460
58465
58470
58475
58480
58485
58490
58495
58500
58505
58510
58515
58520
58525
58530
58535
58540
58545
58550
58555
58560
58565
58570
58575
58580
58585
58590
58595
58600
58605
58610
58615
58620
58625
58630
58635
58640
58645
58650
58655
58660
58665
58670
58675
58680
58685
58690
58695
58700
58705
58710
58715
58720
58725
58730
58735
58740
58745
58750
58755
58760
58765
58770
58775
58780
58785
58790
58795
58800
58805
58810
58815
58820
58825
58830
58835
58840
58845
58850
58855
5886

64860
64865
64870
64875
64880
64885
64890
64895
64900
64905
64910
64915
64920
64925
64930
64935
64940
64945
64950
64955
64960
64965
64970
64975
64980
64985
64990
64995
65000
65005
65010
65015
65020
65025
65030
65035
65040
65045
65050
65055
65060
65065
65070
65075
65080
65085
65090
65095
65100
65105
65110
65115
65120
65125
65130
65135
65140
65145
65150
65155
65160
65165
65170
65175
65180
65185
65190
65195
65200
65205
65210
65215
65220
65225
65230
65235
65240
65245
65250
65255
65260
65265
65270
65275
65280
65285
65290
65295
65300
65305
65310
65315
65320
65325
65330
65335
65340
65345
65350
65355
65360
65365
65370
65375
65380
65385
65390
65395
65400
65405
65410
65415
65420
65425
65430
65435
65440
65445
65450
65455
65460
65465
65470
65475
65480
65485
65490
65495
65500
65505
65510
65515
65520
65525
65530
65535
65540
65545
65550
65555
65560
65565
65570
65575
65580
65585
65590
65595
65600
65605
65610
65615
65620
65625
65630
65635
65640
65645
65650
65655
65660
65665
65670
65675
65680
65685
6569

...

918880
918885
918890
918895
918900
918905
918910
918915
918920
918925
918930
918935
918940
918945
918950
918955
918960
918965
918970
918975
918980
918985
918990
918995
919000
919005
919010
919015
919020
919025
919030
919035
919040
919045
919050
919055
919060
919065
919070
919075
919080
919085
919090
919095
919100
919105
919110
919115
919120
919125
919130
919135
919140
919145
919150
919155
919160
919165
919170
919175
919180
919185
919190
919195
919200
919205
919210
919215
919220
919225
919230
919235
919240
919245
919250
919255
919260
919265
919270
919275
919280
919285
919290
919295
919300
919305
919310
919315
919320
919325
919330
919335
919340
919345
919350
919355
919360
919365
919370
919375
919380
919385
919390
919395
919400
919405
919410
919415
919420
919425
919430
919435
919440
919445
919450
919455
919460
919465
919470
919475
919480
919485
919490
919495
919500
919505
919510
919515
919520
919525
919530
919535
919540
919545
919550
919555
919560
919565
919570
919575
919580
919585
919590

924735
924740
924745
924750
924755
924760
924765
924770
924775
924780
924785
924790
924795
924800
924805
924810
924815
924820
924825
924830
924835
924840
924845
924850
924855
924860
924865
924870
924875
924880
924885
924890
924895
924900
924905
924910
924915
924920
924925
924930
924935
924940
924945
924950
924955
924960
924965
924970
924975
924980
924985
924990
924995
925000
925005
925010
925015
925020
925025
925030
925035
925040
925045
925050
925055
925060
925065
925070
925075
925080
925085
925090
925095
925100
925105
925110
925115
925120
925125
925130
925135
925140
925145
925150
925155
925160
925165
925170
925175
925180
925185
925190
925195
925200
925205
925210
925215
925220
925225
925230
925235
925240
925245
925250
925255
925260
925265
925270
925275
925280
925285
925290
925295
925300
925305
925310
925315
925320
925325
925330
925335
925340
925345
925350
925355
925360
925365
925370
925375
925380
925385
925390
925395
925400
925405
925410
925415
925420
925425
925430
925435
925440
925445

930590
930595
930600
930605
930610
930615
930620
930625
930630
930635
930640
930645
930650
930655
930660
930665
930670
930675
930680
930685
930690
930695
930700
930705
930710
930715
930720
930725
930730
930735
930740
930745
930750
930755
930760
930765
930770
930775
930780
930785
930790
930795
930800
930805
930810
930815
930820
930825
930830
930835
930840
930845
930850
930855
930860
930865
930870
930875
930880
930885
930890
930895
930900
930905
930910
930915
930920
930925
930930
930935
930940
930945
930950
930955
930960
930965
930970
930975
930980
930985
930990
930995
931000
931005
931010
931015
931020
931025
931030
931035
931040
931045
931050
931055
931060
931065
931070
931075
931080
931085
931090
931095
931100
931105
931110
931115
931120
931125
931130
931135
931140
931145
931150
931155
931160
931165
931170
931175
931180
931185
931190
931195
931200
931205
931210
931215
931220
931225
931230
931235
931240
931245
931250
931255
931260
931265
931270
931275
931280
931285
931290
931295
931300

936445
936450
936455
936460
936465
936470
936475
936480
936485
936490
936495
936500
936505
936510
936515
936520
936525
936530
936535
936540
936545
936550
936555
936560
936565
936570
936575
936580
936585
936590
936595
936600
936605
936610
936615
936620
936625
936630
936635
936640
936645
936650
936655
936660
936665
936670
936675
936680
936685
936690
936695
936700
936705
936710
936715
936720
936725
936730
936735
936740
936745
936750
936755
936760
936765
936770
936775
936780
936785
936790
936795
936800
936805
936810
936815
936820
936825
936830
936835
936840
936845
936850
936855
936860
936865
936870
936875
936880
936885
936890
936895
936900
936905
936910
936915
936920
936925
936930
936935
936940
936945
936950
936955
936960
936965
936970
936975
936980
936985
936990
936995
937000
937005
937010
937015
937020
937025
937030
937035
937040
937045
937050
937055
937060
937065
937070
937075
937080
937085
937090
937095
937100
937105
937110
937115
937120
937125
937130
937135
937140
937145
937150
937155

942300
942305
942310
942315
942320
942325
942330
942335
942340
942345
942350
942355
942360
942365
942370
942375
942380
942385
942390
942395
942400
942405
942410
942415
942420
942425
942430
942435
942440
942445
942450
942455
942460
942465
942470
942475
942480
942485
942490
942495
942500
942505
942510
942515
942520
942525
942530
942535
942540
942545
942550
942555
942560
942565
942570
942575
942580
942585
942590
942595
942600
942605
942610
942615
942620
942625
942630
942635
942640
942645
942650
942655
942660
942665
942670
942675
942680
942685
942690
942695
942700
942705
942710
942715
942720
942725
942730
942735
942740
942745
942750
942755
942760
942765
942770
942775
942780
942785
942790
942795
942800
942805
942810
942815
942820
942825
942830
942835
942840
942845
942850
942855
942860
942865
942870
942875
942880
942885
942890
942895
942900
942905
942910
942915
942920


In [3]:
print(getChessManual('u',"0000002"))

('rnbakabnr/9/1c5c1/p1p1p1p1p/9/9/P1P1P1P1P/1C5C1/9/RNBAKABNR r', '全局', '黑胜', [['c八Z五', 0], ['c2Z5', 1], ['n八j七', 0], ['n2j3', 1], ['r九Z八', 0], ['p3j1', 1], ['P三j一', 0], ['c8j4', 1], ['n二j三', 0], ['n8j7', 1], ['n三j四', 0], ['c8Z3', 1], ['c二Z三', 0], ['r9Z8', 1], ['n四j三', 0], ['c5Z6', 1], ['P三j一', 0], ['r8j6', 1], ['B三j一', 0], ['r1Z2', 1], ['r八j九', 0], ['c3j3', 1], ['A六j五', 0], ['n3t2', 1], ['c五j四', 0], ['c6Z3', 1], ['n七j六', 0], ['n7j5', 1], ['n六j五', 0], ['b7j5', 1], ['n五t四', 0], ['r8Z5', 1], ['r一Z二', 0], ['b5j7', 1], ['r二j三', 0], ['r5Z8', 1], ['n四t二', 0], ['c3Z2', 1], ['c三Z八', 0], ['b7t5', 1], ['B一t三', 0], ['c2Z1', 1], ['c八Z五', 0], ['a6j5', 1], ['n三j二', 0], ['c3t3', 1], ['c五j三', 0], ['c3Z6', 1], ['B三j五', 0], ['c6t6', 1], ['n二j四', 0], ['n2j3', 1], ['n二t一', 0], ['c1j4', 1], ['n一t三', 0], ['c6j4', 1], ['n三j四', 0], ['k5Z6', 1], ['n四t二', 0], ['c1t2', 1], ['n二t四', 0], ['c1Z5', 1], ['n四t六', 0], ['c5j1', 1], ['n四j六', 0], ['c5Z4', 1], ['n六j四', 0], ['p1j1', 1], ['A五j四', 0], ['p1j1', 1], ['A四j五', 0]

In [44]:
uname

'u'

In [37]:
b'c2Z5'.decode('gbk')

'c2Z5'

In [46]:
def crawl(url):
    response = urlopen(url)
    # time.sleep(0.1)             # slightly delay for downloading
    return response.read().decode()


def parse(html):
    soup = BeautifulSoup(html, 'lxml')
    urls = soup.find_all('a', {"href": re.compile('^/.+?/$')})
    title = soup.find('h1').get_text().strip()
    page_urls = set([urljoin(base_url, url['href']) for url in urls])   # 去重
    url = soup.find('meta', {'property': "og:url"})['content']
    return title, page_urls, url

RuntimeError: This event loop is already running